ТПИС

In [ ]:
# Установите зависимости (в Colab выполняйте одну ячейку)
!pip install -q sentence-transformers faiss-cpu transformers torch wikipedia==1.4.0

# Прогрессбар (необязательно)
!pip install -q tqdm

In [ ]:
from sentence_transformers import SentenceTransformer
import faiss
import numpy as np
from transformers import pipeline
import wikipedia
import re
from tqdm.auto import tqdm

In [ ]:
# Инициализация моделей (будет загружено из HF)
print("Loading embedding model (sentence-transformers/all-MiniLM-L6-v2)...")
embed_model = SentenceTransformer("all-MiniLM-L6-v2")  # быстрый на CPU

print("Loading NLI model (roberta-large-mnli) as transformers pipeline...")
nli = pipeline("text-classification", model="roberta-large-mnli", device=-1)  # CPU: device=-1

print("Models loaded.")

Loading embedding model (sentence-transformers/all-MiniLM-L6-v2)...


/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md: 0.00B [00:00, ?B/s]

...

vocab.json:   0%|          | 0.00/899k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

Device set to use cpu


Models loaded.


In [ ]:
# Сбор KB: задайте список тем (seed topics). Colab имеет интернет, wikipedia библиотека скачает страницы.
seed_topics = [
  "Albert Einstein", "World War II", "Python (programming language)",
  "Artificial intelligence", "COVID-19", "Barack Obama",
  "Linux", "Microsoft", "Climate change", "Quantum mechanics",
  "New York City", "Mount Everest", "Tesla, Inc.", "Amazon (company)",
  "Google", "Neural network", "Machine learning", "Data science"
]

# Функция: для каждой темы берём summary и делим на фрагменты (~200 tokens ~ ~150-300 chars)
def build_kb_snippets(topics, max_sentences_per_page=15):
    docs = []
    for topic in topics:
        try:
            page = wikipedia.page(topic, auto_suggest=False)
            text = page.content
        except Exception as e:
            try:
                text = wikipedia.summary(topic, auto_suggest=True)
            except Exception as e2:
                print(f"Warning: could not fetch {topic}: {e2}")
                continue
        # split into sentences and group into snippets ~ 2-3 sentences
        sentences = re.split(r'(?<=[.!?])\s+', text)
        cur = []
        for s in sentences[: max_sentences_per_page * 3]:
            cur.append(s.strip())
            if len(cur) >= 3:
                snippet = " ".join(cur)
                docs.append({"title": topic, "snippet": snippet, "source": page.url if 'page' in locals() else "wikipedia:"+topic})
                cur = []
        if cur:
            docs.append({"title": topic, "snippet": " ".join(cur), "source": page.url if 'page' in locals() else "wikipedia:"+topic})
    return docs

print("Building KB snippets (this may take some time)...")
kb_docs = build_kb_snippets(seed_topics)
print(f"KB built: {len(kb_docs)} snippets")

Building KB snippets (this may take some time)...
KB built: 265 snippets


In [ ]:
# Получаем embeddings для snippets
snippets = [d["snippet"] for d in kb_docs]
print("Encoding KB snippets (embedding)...")
embeddings = embed_model.encode(snippets, show_progress_bar=True, convert_to_numpy=True)

# Построим FAISS индекс (cosine via normalized inner product)
d = embeddings.shape[1]
index = faiss.IndexFlatIP(d)  # inner product
# normalize vectors
faiss.normalize_L2(embeddings)
index.add(embeddings)
print(f"FAISS index has {index.ntotal} vectors.")

Encoding KB snippets (embedding)...


Batches:   0%|          | 0/9 [00:00<?, ?it/s]

FAISS index has 265 vectors.


In [ ]:
# 1) claim extraction: простая эвристика — разбить на предложения и выбрать предложения с глаголами/цифрами/именами (lightweight)
sentence_splitter_re = re.compile(r'(?<=[.!?])\s+')
def extract_claims(text, min_len=30):
    sents = sentence_splitter_re.split(text)
    candidates = []
    for s in sents:
        s = s.strip()
        if len(s) < min_len:
            continue
        # простая filter: contains digit or proper noun (capitalized) or 'is/was/are' — heuristic to catch factual claims
        if re.search(r'\b(is|was|are|won|died|born|founded|established|announced|reported|has|have)\b', s, re.I) or re.search(r'\d', s):
            candidates.append(s)
    # fallback: if none, take longest sentences
    if not candidates:
        candidates = sorted([s for s in sents if len(s) >= min_len], key=len, reverse=True)[:3]
    return candidates[:8]  # limit to 8 claims

# 2) retrieval: embed claim, search FAISS, rerank by cosine on original embedding (we used IP with normalized vectors)
def retrieve_proofs(claim_text, top_k=10):
    emb = embed_model.encode([claim_text], convert_to_numpy=True)
    faiss.normalize_L2(emb)
    D, I = index.search(emb, top_k)
    results = []
    for i, score in zip(I[0], D[0]):
        results.append({
            "snippet": snippets[i],
            "source": kb_docs[i]["source"],
            "score": float(score)
        })
    return results

# 3) NLI verification: for each (claim, snippet) check entailment probability
def nli_score(claim, snippet):
    # roberta-large-mnli returns labels: CONTRADICTION / NEUTRAL / ENTAILMENT
    hypothesis = claim
    premise = snippet
    res = nli(f"{premise} </s></s> {hypothesis}")
    # res is list of dicts with label and score
    # Find entailment score
    # For pipeline text-classification with roberta-large-mnli, result may be multi-entry; handle generically
    if isinstance(res, list):
        # pipeline returns list; find entailment
        ent = None
        for r in res:
            if r['label'].upper().startswith('ENTAIL'):
                ent = r['score']
        if ent is None:
            # fallback: choose highest score label: if ENTAIL present return that score else 0
            ent = max([r['score'] for r in res])
        return float(ent)
    else:
        # fallback
        return 0.0

# 4) aggregate NLI scores for top proofs and make verdict
def assess_claim(claim_text, top_k=6):
    proofs = retrieve_proofs(claim_text, top_k=top_k)
    # For speed, we will evaluate NLI on top_k (can be heavy)
    scores = []
    best = None
    best_score = -1.0
    for p in proofs:
        s = nli_score(claim_text, p["snippet"])
        p["nli_entailment"] = s
        scores.append(s)
        if s > best_score:
            best_score = s
            best = p
    # aggregate rule: use max entailment as support
    support = max(scores) if scores else 0.0
    return {
        "claim": claim_text,
        "support": support,
        "best_proof": best,
        "all_proofs": proofs
    }

# 5) decision policy: thresholds (tunable)
def decision_from_support(support, allow_t=0.85, escalate_t=0.4):
    if support >= allow_t:
        return "allow"
    elif support <= escalate_t:
        return "escalate"
    else:
        return "warn"

In [ ]:
def analyze_text(llm_text, verbose=True):
    """
    Input: llm_text (string) — текст ответа от LLM (или сгенерированный)
    Output: dict with claims analysis and overall verdicts
    """
    # 1) extract claims
    claims = extract_claims(llm_text)
    results = []
    for c in claims:
        res = assess_claim(c, top_k=6)
        res['decision'] = decision_from_support(res['support'])
        results.append(res)
        if verbose:
            print("CLAIM:", c)
            print("  best support:", res['support'])
            if res['best_proof']:
                print("  best proof snippet:", res['best_proof']['snippet'][:250], "...")
                print("  source:", res['best_proof']['source'])
            print("  decision:", res['decision'])
            print("-"*80)
    # overall policy: if any claim escalated -> escalate response; elif any warn -> warn; else allow
    decisions = [r['decision'] for r in results]
    if "escalate" in decisions:
        overall = "escalate"
    elif "warn" in decisions:
        overall = "warn"
    else:
        overall = "allow"
    return {"claims": results, "overall": overall}

In [ ]:
# Option A: supply an LLM answer manually (recommended for control)
sample_response = (
    "Albert Einstein was born in 1879 in Berlin. "
    "He won the Nobel Prize for Physics in 1921 for the theory of relativity. "
    "Python is a statically typed language. "
    "Mount Everest height is 8848 meters."
)

print("Analyzing provided sample_response...\n")
out = analyze_text(sample_response, verbose=True)
print("Overall verdict:", out["overall"])

# Option B: (optional) use a small local generator to produce text (distilgpt2)
# This is lightweight but not high-quality; uncomment to use.
"""
from transformers import pipeline
gen = pipeline("text-generation", model="distilgpt2")
prompt = "Tell me a few facts about Python and AI."
resp = gen(prompt, max_length=150, do_sample=False)[0]["generated_text"]
print(resp)
analyze_text(resp)
"""

Analyzing provided sample_response...

CLAIM: Albert Einstein was born in 1879 in Berlin.
  best support: 0.9990614056587219
  best proof snippet: As a result, he became increasingly isolated from mainstream modern physics. == Life and career ==


=== Childhood, youth and education ===

Albert Einstein was born in Ulm, in the Kingdom of Württemberg in the German Empire, on 14 March 1879. His pa ...
  source: https://en.wikipedia.org/wiki/Albert_Einstein
  decision: allow
--------------------------------------------------------------------------------
CLAIM: He won the Nobel Prize for Physics in 1921 for the theory of relativity.
  best support: 0.997358500957489
  best proof snippet: Albert Einstein (14 March 1879 – 18 April 1955) was a German-born theoretical physicist best known for developing the theory of relativity. Einstein also made important contributions to quantum theory. His mass–energy equivalence formula E = mc2, whi ...
  source: https://en.wikipedia.org/wiki/Albert_Einst

'\nfrom transformers import pipeline\ngen = pipeline("text-generation", model="distilgpt2")\nprompt = "Tell me a few facts about Python and AI."\nresp = gen(prompt, max_length=150, do_sample=False)[0]["generated_text"]\nprint(resp)\nanalyze_text(resp)\n'

In [ ]:
# Option B: (optional) use a small local generator to produce text (distilgpt2)
# This is lightweight but not high-quality; uncomment to use.
from transformers import pipeline
gen = pipeline("text-generation", model="distilgpt2")
prompt = "Tell me a few facts about Python and AI."
resp = gen(prompt, max_length=150, do_sample=False)[0]["generated_text"]
print(resp)
analyze_text(resp)

Device set to use cpu
Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Both `max_new_tokens` (=256) and `max_length`(=150) seem to have been set. `max_new_tokens` will take precedence. Please refer to the documentation for more information. (https://huggingface.co/docs/transformers/main/en/main_classes/text_generation)


Tell me a few facts about Python and AI.


Python is a Python programming language that is designed to be used for the purpose of learning and learning Python.
Python is a Python programming language that is designed to be used for the purpose of learning and learning Python.
Python is a Python programming language that is designed to be used for the purpose of learning and learning Python.
Python is a Python programming language that is designed to be used for the purpose of learning and learning Python.
Python is a Python programming language that is designed to be used for the purpose of learning and learning Python.
Python is a Python programming language that is designed to be used for the purpose of learning and learning Python.
Python is a Python programming language that is designed to be used for the purpose of learning and learning Python.
Python is a Python programming language that is designed to be used for the purpose of learning and learning Python.
Python is a Python pr

{'claims': [{'claim': 'Python is a Python programming language that is designed to be used for the purpose of learning and learning Python.',
   'support': 0.9342474937438965,
   'best_proof': {'snippet': 'Python uses dynamic typing and a combination of reference counting and a cycle-detecting garbage collector for memory management. It uses dynamic name resolution (late binding), which binds method and variable names during program execution. Python\'s design offers some support for functional programming in the "Lisp tradition".',
    'source': 'https://en.wikipedia.org/wiki/Python_(programming_language)',
    'score': 0.5991578102111816,
    'nli_entailment': 0.9342474937438965},
   'all_proofs': [{'snippet': 'Python is a high-level, general-purpose programming language. Its design philosophy emphasizes code readability with the use of significant indentation. Python is dynamically type-checked and garbage-collected.',
     'source': 'https://en.wikipedia.org/wiki/Python_(programmin

In [ ]:
sample_response = (
    "Albert Einstein was born in 1879 in Berlin. "
    "JavaScript is a typical typed language. "
    "Ana de Armas borin in 1949 year. "
    "Python is a statically typed language. "
)

print("Analyzing provided sample_response...\n")
out = analyze_text(sample_response, verbose=True)
print("Overall verdict:", out["overall"])

Analyzing provided sample_response...

CLAIM: Albert Einstein was born in 1879 in Berlin.
  best support: 0.9990614056587219
  best proof snippet: As a result, he became increasingly isolated from mainstream modern physics. == Life and career ==


=== Childhood, youth and education ===

Albert Einstein was born in Ulm, in the Kingdom of Württemberg in the German Empire, on 14 March 1879. His pa ...
  source: https://en.wikipedia.org/wiki/Albert_Einstein
  decision: allow
--------------------------------------------------------------------------------
CLAIM: JavaScript is a typical typed language.
  best support: 0.9071488976478577
  best proof snippet: Desktop Linux distributions include a windowing system such as X11 or Wayland and a desktop environment such as GNOME, KDE Plasma or Xfce. Distributions intended for servers may not have a graphical user interface at all or include a solution stack s ...
  source: https://en.wikipedia.org/wiki/Linux
  decision: allow
--------------------